# divmod

In [1]:
s = pd.Series(np.arange(10))
s

0    0
1    1
2    2
3    3
4    4
5    5
6    6
7    7
8    8
9    9
dtype: int32

In [2]:
divmod(s,3) # s//3  s%3

(0    0
 1    0
 2    0
 3    1
 4    1
 5    1
 6    2
 7    2
 8    2
 9    3
 dtype: int32, 0    0
 1    1
 2    2
 3    0
 4    1
 5    2
 6    0
 7    1
 8    2
 9    0
 dtype: int32)

In [12]:
idx = pd.Index(np.arange(10))
divmod(idx,3)

(Int64Index([0, 0, 0, 1, 1, 1, 2, 2, 2, 3], dtype='int64'),
 Int64Index([0, 1, 2, 0, 1, 2, 0, 1, 2, 0], dtype='int64'))

# equals

In [13]:
df1 = pd.DataFrame({'col':['foo', 0, np.nan]})
df2 = pd.DataFrame({'col':[np.nan, 0, 'foo']}, index=[2,1,0])

In [17]:
df1.equals(df2)
df1.equals(df2.sort_index())

False

True

# pd.Series(['foo', 'bar', 'baz']) == pd.Series(['foo'])
ValueError: Series lengths must match to compare

In [19]:
pd.Series(['foo', 'bar', 'baz']) == 'foo'

0     True
1    False
2    False
dtype: bool

# combine_first

In [21]:
df1 = pd.DataFrame({'A' : [1., np.nan, 3., 5., np.nan],
                    'B' : [np.nan, 2., 3., np.nan, 6.]})
df2 = pd.DataFrame({'A' : [5., 2., 4., np.nan, 3., 7.],
                    'B' : [np.nan, np.nan, 3., 4., 6., 8.]})

In [23]:
df1
df2

A    B
0  1.0  NaN
1  NaN  2.0
2  3.0  3.0
3  5.0  NaN
4  NaN  6.0

A    B
0  5.0  NaN
1  2.0  NaN
2  4.0  3.0
3  NaN  4.0
4  3.0  6.0
5  7.0  8.0

In [29]:
df1.combine_first(df2)
df1.combine(df2, lambda x,y: np.where(pd.isnull(x),y,x))

A    B
0  1.0  NaN
1  2.0  2.0
2  3.0  3.0
3  5.0  4.0
4  3.0  6.0
5  7.0  8.0

A    B
0  1.0  NaN
1  2.0  2.0
2  3.0  3.0
3  5.0  4.0
4  3.0  6.0
5  7.0  8.0

In [27]:
df1.combine(df2,lambda x,y:x+y)

A     B
0  6.0   NaN
1  NaN   NaN
2  7.0   6.0
3  NaN   NaN
4  NaN  12.0
5  NaN   NaN

# pipe

In [57]:
def mul1(x):
    return x
def mul2(x,y):
    return x+y*2
def mul3(x,y,z):
    return x+y*2+z*3

In [66]:
df = DataFrame({'A':np.random.randint(0,1,7),
                'B':np.random.randint(1,4,7)})
df

A  B
0  0  3
1  0  1
2  0  2
3  0  3
4  0  3
5  0  2
6  0  2

In [80]:
df.pipe(mul1).pipe((mul2,'y'),1)
df.pipe(mul1).pipe((mul2,'y'),1).pipe((mul3,'y'),x=0,z=0)

A  B
0  1  7
1  1  3
2  1  5
3  1  7
4  1  7
5  1  5
6  1  5

A   B
0  2  14
1  2   6
2  2  10
3  2  14
4  2  14
5  2  10
6  2  10

# transform

In [24]:
A=np.array([[1,2,3,4,5],[2,1,1,2,2],[1,2,3,4,5],[2,1,1,2,2],[1,2,3,4,5]])
data=pd.DataFrame(A,index=['li','chen','wang','zhao','qian'],columns=['a','b','c','d','e'])
data

a  b  c  d  e
li    1  2  3  4  5
chen  2  1  1  2  2
wang  1  2  3  4  5
zhao  2  1  1  2  2
qian  1  2  3  4  5

In [29]:
key=['ss','kk','kk','ss','ss']
data.groupby(key).transform(np.sum)
data.groupby(key).agg(sum)

a  b  c   d   e
li    4  5  7  10  12
chen  3  3  4   6   7
wang  3  3  4   6   7
zhao  4  5  7  10  12
qian  4  5  7  10  12

a  b  c   d   e
kk  3  3  4   6   7
ss  4  5  7  10  12

# apply

In [3]:
tsdf = pd.DataFrame(np.random.randn(1000, 3), columns=['A', 'B', 'C'],
                     index=pd.date_range('1/1/2000', periods=1000))
tsdf[:5]

A         B         C
2000-01-01  0.373971  0.928985 -0.196897
2000-01-02  1.088382  0.113991  0.190444
2000-01-03 -0.174084  0.097064  0.108707
2000-01-04  0.910716 -1.133728  0.918753
2000-01-05  1.365255  1.149368 -1.152388

In [5]:
def subtract_and_divide(x, sub, divide=1):
    return (x - sub) / divide
tsdf[:5].apply(subtract_and_divide,args=(5,),divide=1)
tsdf[:5].apply(subtract_and_divide,args=(5,1))

A         B         C
2000-01-01 -4.626029 -4.071015 -5.196897
2000-01-02 -3.911618 -4.886009 -4.809556
2000-01-03 -5.174084 -4.902936 -4.891293
2000-01-04 -4.089284 -6.133728 -4.081247
2000-01-05 -3.634745 -3.850632 -6.152388

A         B         C
2000-01-01 -4.626029 -4.071015 -5.196897
2000-01-02 -3.911618 -4.886009 -4.809556
2000-01-03 -5.174084 -4.902936 -4.891293
2000-01-04 -4.089284 -6.133728 -4.081247
2000-01-05 -3.634745 -3.850632 -6.152388

# partial

In [6]:
mdf = pd.DataFrame({'A': [1, 2, 3],
                     'B': [1., 2., 3.],
                     'C': ['foo', 'bar', 'baz'],
                     'D': pd.date_range('20130101', periods=3)})

mdf

A    B    C          D
0  1  1.0  foo 2013-01-01
1  2  2.0  bar 2013-01-02
2  3  3.0  baz 2013-01-03

In [11]:
from functools import partial
q25 = partial(Series.quantile,q=0.25)
q25.__name__ = '25%'
mdf.apply(['sum','min',q25,lambda x : Series.quantile(x,q=0.25)])

A    B          C                   D
25%       1.5  1.5        NaN 2013-01-01 12:00:00
<lambda>  1.5  1.5        NaN 2013-01-01 12:00:00
min       1.0  1.0        bar 2013-01-01 00:00:00
sum       6.0  6.0  foobarbaz                 NaT

# align

In [14]:
s = pd.Series(np.random.randn(5), index=['a', 'b', 'c', 'd', 'e'])
s1 = s[:4]
s2 = s[1:]

In [15]:
s1.align(s2)

(a   -2.400398
 b   -0.431470
 c   -0.228160
 d   -0.043043
 e         NaN
 dtype: float64, a         NaN
 b   -0.431470
 c   -0.228160
 d   -0.043043
 e   -0.031154
 dtype: float64)

In [17]:
s1.align(s2,join='left')

(a   -2.400398
 b   -0.431470
 c   -0.228160
 d   -0.043043
 dtype: float64, a         NaN
 b   -0.431470
 c   -0.228160
 d   -0.043043
 dtype: float64)

# Iteration

In [21]:
for i in s.iteritems():
    print(i)

('a', -2.4003979037567369)
('b', -0.43147021601955887)
('c', -0.22816025576692825)
('d', -0.043043139905031461)
('e', -0.031153949508241761)


In [24]:
for i in mdf.iterrows():
    print(i)

(0, A                      1
B                      1
C                    foo
D    2013-01-01 00:00:00
Name: 0, dtype: object)
(1, A                      2
B                      2
C                    bar
D    2013-01-02 00:00:00
Name: 1, dtype: object)
(2, A                      3
B                      3
C                    baz
D    2013-01-03 00:00:00
Name: 2, dtype: object)


In [25]:
for i in mdf.iteritems():
    print(i)

('A', 0    1
1    2
2    3
Name: A, dtype: int64)
('B', 0    1.0
1    2.0
2    3.0
Name: B, dtype: float64)
('C', 0    foo
1    bar
2    baz
Name: C, dtype: object)
('D', 0   2013-01-01
1   2013-01-02
2   2013-01-03
Name: D, dtype: datetime64[ns])


In [26]:
for i in mdf.itertuples():
    print(i)

Pandas(Index=0, A=1, B=1.0, C='foo', D=Timestamp('2013-01-01 00:00:00'))
Pandas(Index=1, A=2, B=2.0, C='bar', D=Timestamp('2013-01-02 00:00:00'))
Pandas(Index=2, A=3, B=3.0, C='baz', D=Timestamp('2013-01-03 00:00:00'))


# .dt accessor

In [27]:
s = pd.Series(pd.date_range('20130101 09:10:12', periods=4))
s

0   2013-01-01 09:10:12
1   2013-01-02 09:10:12
2   2013-01-03 09:10:12
3   2013-01-04 09:10:12
dtype: datetime64[ns]

In [30]:
s.dt.hour

0    9
1    9
2    9
3    9
dtype: int64

In [31]:
s.dt.day

0    1
1    2
2    3
3    4
dtype: int64

In [40]:
stz = s.dt.tz_localize('US/Eastern')
stz

0   2013-01-01 09:10:12-05:00
1   2013-01-02 09:10:12-05:00
2   2013-01-03 09:10:12-05:00
3   2013-01-04 09:10:12-05:00
dtype: datetime64[ns, US/Eastern]

In [41]:
stz.dt.tz

<DstTzInfo 'US/Eastern' LMT-1 day, 19:04:00 STD>

In [44]:
stc = s.dt.tz_localize('Asia/Shanghai')
stc
stc.dt.tz

0   2013-01-01 09:10:12+08:00
1   2013-01-02 09:10:12+08:00
2   2013-01-03 09:10:12+08:00
3   2013-01-04 09:10:12+08:00
dtype: datetime64[ns, Asia/Shanghai]

<DstTzInfo 'Asia/Shanghai' LMT+8:06:00 STD>

In [1]:
s = pd.Series(pd.timedelta_range('1 day 00:00:05', periods=4, freq='s'))
s

0   1 days 00:00:05
1   1 days 00:00:06
2   1 days 00:00:07
3   1 days 00:00:08
dtype: timedelta64[ns]

In [6]:
s.dt.days

0    1
1    1
2    1
3    1
dtype: int64

In [7]:
s.dt.components

days  hours  minutes  seconds  milliseconds  microseconds  nanoseconds
0     1      0        0        5             0             0            0
1     1      0        0        6             0             0            0
2     1      0        0        7             0             0            0
3     1      0        0        8             0             0            0

In [16]:
df1 = pd.DataFrame({'one':[2,1,1,1],'two':[1,3,2,4],'three':[5,4,3,2]},index=['b','d','a','c'])

In [17]:
df1

one  three  two
b    2      5    1
d    1      4    3
a    1      3    2
c    1      2    4

In [18]:
df1.index.name='letter'

In [19]:
df1

one  three  two
letter                 
b         2      5    1
d         1      4    3
a         1      3    2
c         1      2    4

In [21]:
df1.sort_index().sort_values(by=['one'])

one  three  two
letter                 
a         1      3    2
c         1      2    4
d         1      4    3
b         2      5    1

# searchsorted¶

In [22]:
ser = pd.Series([1, 2, 3])
ser

0    1
1    2
2    3
dtype: int64

In [23]:
ser.searchsorted([0,1])

array([0, 0], dtype=int64)

In [24]:
ser.searchsorted([0,5])

array([0, 3], dtype=int64)

In [27]:
ser.searchsorted([0,2],side='right')

array([0, 2], dtype=int64)

In [28]:
ser.searchsorted([0,1,2],side='right')

array([0, 1, 2], dtype=int64)

In [29]:
ser.searchsorted([0,1,2,11],side='right')

array([0, 1, 2, 3], dtype=int64)

# nlagest nsmallest

In [32]:
df = pd.DataFrame({'a': [-2, -1, 1, 10, 8, 11, -1],
                    'b': list('abdceff'),
                    'c': [1.0, 2.0, 4.0, 3.2, np.nan, 3.0, 4.0]})
df

a  b    c
0  -2  a  1.0
1  -1  b  2.0
2   1  d  4.0
3  10  c  3.2
4   8  e  NaN
5  11  f  3.0
6  -1  f  4.0

In [37]:
df.nlargest(columns=['a','c'],n=3)

a  b    c
5  11  f  3.0
3  10  c  3.2
4   8  e  NaN

In [43]:
df.nsmallest(columns=['a'],n=5)
df.nsmallest(columns=['a','c'],n=5)

a  b    c
0 -2  a  1.0
1 -1  b  2.0
6 -1  f  4.0
2  1  d  4.0
4  8  e  NaN

a  b    c
0 -2  a  1.0
2  1  d  4.0
4  8  e  NaN
1 -1  b  2.0
6 -1  f  4.0

In [54]:
df
df.nsmallest(columns=['a','c'],n=5)

a  b    c
0  -2  a  1.0
1  -1  b  2.0
2   1  d  4.0
3  10  c  3.2
4   8  e  NaN
5  11  f  3.0
6  -1  f  4.0

a  b    c
0 -2  a  1.0
2  1  d  4.0
4  8  e  NaN
1 -1  b  2.0
6 -1  f  4.0

In [55]:
df1

one  three  two
letter                 
b         2      5    1
d         1      4    3
a         1      3    2
c         1      2    4